In [12]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score

In [2]:
with open("all_test.posteriors_new3") as kaldi_posteriors_file:
    kaldi_posteriors = kaldi_posteriors_file.readlines()

In [3]:
posteriors = pd.DataFrame(columns=["speaker_id", "sentence_id", "frame_number", "use", "kaldi_posteriors", "label", "mapped_label"])

In [4]:
frame_number = 1
use = "test"
dummy_np = []
for line in kaldi_posteriors:
    line = re.sub(" \[", "", line.strip())
    line = re.sub(" \]", "", line.strip())
    if("_" in line):
        speaker_id, sentence_id = line.split("_")
        frame_number = 1
    else:
        posteriors = [float(ele) for ele in line.split(" ")]
        dummy_np.append([speaker_id, sentence_id, frame_number, use, posteriors, "None"])
        frame_number += 1

In [5]:
kaldi_labels = pd.read_pickle("test_with_mapped_labels.pkl")

In [6]:
kaldi_labels.head()

,speaker_id,sentence_id,frame_number,use,feature_vector,label,mapped_labels
0,FDHC0,SI1559,1,train,"[-4.52633, 0.02125172, -3.103144, -0.1826605, ...",0,42
1,FDHC0,SI1559,2,train,"[-5.009317, -0.1809075, -3.107169, -0.1852918,...",0,42
2,FDHC0,SI1559,3,train,"[-5.625691, -0.3789468, -3.112408, -0.01292514...",0,42
3,FDHC0,SI1559,4,train,"[-6.03809, -0.3803073, -3.133071, -0.5771251, ...",50,41
4,FDHC0,SI1559,5,train,"[-6.048991, -1.165254, -2.996357, -1.239447, 0...",50,41


In [7]:
speaker_ids = [ele[0] for ele in dummy_np]
sentence_ids = [ele[1] for ele in dummy_np]
frame_numbers = [ele[2] for ele in dummy_np]
use = [ele[3] for ele in dummy_np]
posteriors = [ele[4] for ele in dummy_np]
label = kaldi_labels["label"]

In [8]:
posteriors[0].index(max(posteriors[0]))

0

In [9]:
def get_map_1944_to_48(id):
    phone_maps = pd.read_pickle("phone_map_1944_to_48.pkl")
    phones = phone_maps["phone"]
    maps = phone_maps["map"]
    for i in range(len(maps)):
        if(id in maps[i]):
            return(i)

In [10]:
kaldi_labels_list = []
for i in range(len(posteriors)):
    kaldi_labels_list.append(posteriors[i].index(max(posteriors[i])))

In [11]:
mapped_kaldi_phones = [get_map_1944_to_48(ele) for ele in kaldi_labels_list]

In [20]:
accuracy_score(kaldi_labels["mapped_labels"].tolist(), mapped_kaldi_phones)

0.030093751618639825

In [29]:
len(set(kaldi_labels_list)&set(kaldi_labels["label"])) / float(len(set(kaldi_labels_list) | set(kaldi_labels["label"]))) * 100

81.35942327497425

In [30]:
len(set(mapped_kaldi_phones)&set(kaldi_labels["mapped_labels"])) / float(len(set(mapped_kaldi_phones) | set(kaldi_labels["mapped_labels"]))) * 100

100.0

In [31]:
81.07271789582259

81.07271789582259